In [426]:
PIANO_GENERATOR_MODEL_WEIGHTS_PATH = './piano models/piano_model_weights'
DRUMS_GENERATOR_MODEL_WEIGHTS_PATH = './model02'

## Piano generator

In [10]:
from datetime import datetime
import itertools
import json
import os
import shutil

import os

import glob
import pickle
import numpy
import pandas
from pydub import AudioSegment
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from keras import backend as K
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
import numpy as np

from data import *
from midi_util import array_to_midi, print_array

In [2]:
def load_learned_network(network_input, n_vocab, weights_path):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    # Load the weights to each node
    model.load_weights(weights_path)

    return model

In [3]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(200):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [388]:
def create_midi(prediction_output, 
                output_path):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
#         offset += 0.5
        offset += 1.75

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=output_path)

In [389]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 32
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [390]:
def generate_piano_roll(weights_path, output_path):
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = load_learned_network(normalized_input, n_vocab, weights_path)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output, output_path)

In [68]:
generate_piano_roll(weights_path=PIANO_GENERATOR_MODEL_WEIGHTS_PATH + '_01', output_path='piano_roll_08.mid')

In [69]:
os.system('fluidsynth -ni Touhou.sf2 piano_roll_08.mid -F piano_roll_08.wav -r 44100')

0

## Drums generator

In [21]:
# All the pitches represented in the MIDI data arrays.
# TODO: Read pitches from pitches.txt file in corresponding midi array
# directory.
PITCHES = [36, 37, 38, 40, 41, 42, 44, 45, 46, 47, 49, 50, 58, 59, 60, 61, 62, 63, 64, 66]
# The subset of pitches we'll actually use.
IN_PITCHES = [36, 38, 42, 58, 59, 61]#[36, 38, 41, 42, 47, 58, 59, 61]
# The pitches we want to generate (potentially for different drum kit)
OUT_PITCHES = IN_PITCHES#[54, 56, 58, 60, 61, 62, 63, 64]
# The minimum number of hits to keep a drum loop after the types of
# hits have been filtered by IN_PITCHES.
MIN_HITS = 8

In [22]:
NUM_HIDDEN_UNITS = 128
# The length of the phrase from which the predict the next symbol.
PHRASE_LEN = 64
# Dimensionality of the symbol space.
SYMBOL_DIM = 2 ** len(IN_PITCHES)
NUM_ITERATIONS = 2
BATCH_SIZE = 128

# VALIDATION_PERCENT = 0.1
VALIDATION_PERCENT = 0.001

BASE_DIR = './'
#BASE_DIR = '/home/ubuntu/neural-beats'

#MIDI_IN_DIR = os.path.join(BASE_DIR, 'midi_arrays/mega/')
#MIDI_IN_DIR = os.path.join(BASE_DIR, 'midi_arrays/mega/Electronic Live 9 SD/Jungle')
MIDI_IN_DIR = os.path.join(BASE_DIR, 'drums midi')
#MIDI_IN_DIR = os.path.join(BASE_DIR, 'midi_arrays/mega/Rock Essentials 2 Live 9 SD/Preview Files/Fills/4-4 Fills')

MODEL_OUT_DIR = os.path.join(BASE_DIR, 'models')
MODEL_NAME = 'drum_generator'
TRIAL_DIR = os.path.join(MODEL_OUT_DIR, MODEL_NAME)

MIDI_OUT_DIR = os.path.join(TRIAL_DIR, 'gen-midi')

In [34]:
encodings = {
    config : i
    for i, config in enumerate(itertools.product([0,1], repeat=len(IN_PITCHES)))
}

def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = a.astype('float64')
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))


def encode(midi_array):
    '''Encode a folded MIDI array into a sequence of integers.'''
    return [
        encodings[tuple((time_slice>0).astype(int))]
        for time_slice in midi_array
    ]

decodings = {
    i : config
    for i, config in enumerate(itertools.product([0,1], repeat=len(IN_PITCHES)))
}

def decode(config_ids):
    '''Decode a sequence of integers into a folded MIDI array.'''
    velocity = 120
#     velocity = 1
    return velocity * np.vstack(
        [list(decodings[id]) for id in config_ids])

In [32]:
def unfold(midi_array, pitches):
    '''Unfold a folded MIDI array with the given pitches.'''
    # Create an array of all the 128 pitches and fill in the
    # corresponding pitches.
    res = np.zeros((midi_array.shape[0], 128))
    assert midi_array.shape[1] == len(pitches), 'Mapping between unequal number of pitches!'
    for i in range(len(pitches)):
        res[:,pitches[i]] = midi_array[:,i]
    return res

In [18]:
def prepare_data():
    # Load the data.
    # Concatenate all the vectorized midi files.
    num_steps = 0

    # Sequence of configuration numbers representing combinations of
    # active pitches.
    config_sequences = []
    num_dirs = len([x for x in os.walk(MIDI_IN_DIR)])
    assert num_dirs > 0, 'No data found at {}'.format(MIDI_IN_DIR)

    in_pitch_indices = [ PITCHES.index(p) for p in IN_PITCHES ]
    for dir_idx, (root, dirs, files) in enumerate(os.walk(MIDI_IN_DIR)):
        for filename in files:
            if filename.split('.')[-1] != 'npy':
                continue
            array = np.load(os.path.join(root, filename))
            if np.sum(np.sum(array[:, in_pitch_indices]>0)) < MIN_HITS:
                continue
            config_sequences.append(np.array(encode(array[:, in_pitch_indices])))
        print('Loaded {}/{} directories'.format(dir_idx + 1, num_dirs))

    # Construct labeled examples.
    # Use a generator for X and y as the whole dataset may not fit in
    # memory.
#     train_generator = SequenceDataGenerator(config_sequences,
#                                             phrase_length=PHRASE_LEN,
#                                             dim=SYMBOL_DIM,
#                                             batch_size=BATCH_SIZE,
#                                             is_validation=False,
#                                             validation_percent=VALIDATION_PERCENT)

#     valid_generator = SequenceDataGenerator(config_sequences,
#                                             phrase_length=PHRASE_LEN,
#                                             dim=SYMBOL_DIM,
#                                             batch_size=BATCH_SIZE,
#                                             is_validation=True,
#                                             validation_percent=VALIDATION_PERCENT)

    return config_sequences

In [13]:
def init_model():
    # Build the model.
    model = Sequential()
    model.add(LSTM(
        NUM_HIDDEN_UNITS,
        return_sequences=True,
        input_shape=(PHRASE_LEN, SYMBOL_DIM)))
    model.add(Dropout(0.3))
    '''
    model.add(LSTM(
        NUM_HIDDEN_UNITS,
        return_sequences=True,
        input_shape=(SYMBOL_DIM, SYMBOL_DIM)))
    model.add(Dropout(0.2))
    '''
    model.add(LSTM(NUM_HIDDEN_UNITS, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(SYMBOL_DIM))
    model.add(Activation('softmax'))
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(learning_rate=1e-03, rho=0.9, epsilon=1e-08))
    return model

In [14]:
def generate(model, seed, mid_name, temperature=1.0, length=512, tpq=1000):
    '''Generate sequence using model, seed, and temperature.'''

    generated = []
    phrase = seed

    if not hasattr(temperature, '__len__'):
        temperature = [temperature for _ in range(length)]

    for temp in temperature:
        x = np.zeros((1, PHRASE_LEN, SYMBOL_DIM))
        for t, config_id in enumerate(phrase):
            x[0, t, config_id] = 1
        preds = model.predict(x, verbose=0)[0]
        next_id = sample(preds, temp)

        generated += [next_id]
        phrase = phrase[1:] + [next_id]

    # ticks per quarter has negative correlation with drums speed
    mid = array_to_midi(unfold(decode(generated), OUT_PITCHES), mid_name, ticks_per_quarter=tpq)
    mid.save(os.path.join(MIDI_OUT_DIR, mid_name))
    return mid

In [15]:
def generate_drums(model, config_sequences, output_path):
    sequence_indices = idx_seq_of_length(config_sequences, PHRASE_LEN)
    seq_index, phrase_start_index = sequence_indices[
                np.random.choice(len(sequence_indices))]
    gen_length = 512
    for temperature in [1.0, 0.5, 0.75, 1.0]:
        generated = []
        phrase = list(
            config_sequences[seq_index][
                phrase_start_index: phrase_start_index + PHRASE_LEN
            ]
        )

        print('----- Generating with temperature:', temperature)

        midi = generate(model,
                        phrase,
                        'out_{}_{}_{}.mid'.format(gen_length, temperature, 0),
                        temperature=temperature,
                        length=gen_length)
        break
        
    midi.save(output_path)

In [26]:
config_sequences = prepare_data()

Loaded 1/1 directories


In [29]:
drums_model = init_model()
drums_model.load_weights(DRUMS_GENERATOR_MODEL_WEIGHTS_PATH)

In [35]:
generate_drums(drums_model, config_sequences, 'drums_01.mid')

----- Generating with temperature: 1.0


<ipython-input-34-45a54c43edaa>:8: RuntimeWarning: divide by zero encountered in log
  a = np.log(a) / temperature


In [36]:
os.system('fluidsynth -ni Touhou.sf2 drums_01.mid -F drums_01.wav -r 44100')

0

## Generate Piano roll & Drums line

In [391]:
def generate_lines():
    # generate piano roll midi file
    generate_piano_roll(weights_path=PIANO_GENERATOR_MODEL_WEIGHTS_PATH + '_01', 
                        output_path='piano_roll.mid')
    # convert midi to wav format
    os.system('fluidsynth -ni Touhou.sf2 piano_roll.mid -F piano_roll.wav -r 44100')
    # delete piano roll midi file
    os.system('rm -f piano_roll.mid')
    
    # generate drums line midi file
    config_sequences = prepare_data()
    drums_model = init_model()
    drums_model.load_weights(DRUMS_GENERATOR_MODEL_WEIGHTS_PATH)
    generate_drums(drums_model, config_sequences, 'drums_line.mid')
    # convert midi to wav format
    os.system('fluidsynth -ni Touhou.sf2 drums_line.mid -F drums_line.wav -r 44100')
    # delete drums line midi file
    os.system('rm -f drums_line.mid')

In [394]:
generate_lines()

Loaded 1/1 directories
----- Generating with temperature: 1.0


<ipython-input-34-45a54c43edaa>:8: RuntimeWarning: divide by zero encountered in log
  a = np.log(a) / temperature


## Generate Lo-Fi

In [428]:
SECOND = 1000

In [429]:
def mix_lines(drums_line, piano_line, noise_line, vocal_line, music_len=60):
    drums_line = drums_line[5*SECOND : (music_len + 5)*SECOND]
    piano_line = piano_line[: music_len*SECOND]
    noise_line = noise_line[: music_len*SECOND]
    
    piano = piano_line.low_pass_filter(40)
    piano = piano + 12
    # piano = piano.low_pass_filter(110)
    drums = drums_line.low_pass_filter(95)
    drums = drums + 22
    noise = noise_line - 10
    vocal = vocal_line + 5
    
    piano = piano.fade_in(4*SECOND)
    noise = noise.fade_in(10*SECOND)
    
    music = noise.overlay(
        vocal, 
        position=4*SECOND
    ).overlay(
        piano, 
        position=10*SECOND
    ).overlay(
        drums, 
        position=15*SECOND
    )
    
    music = music.fade_out(8*SECOND)
    
    return music

In [82]:
SECOND = 1000

In [395]:
drums_line = AudioSegment.from_wav('drums_line.wav')
piano_line = AudioSegment.from_wav('piano_roll.wav')

In [396]:
noise_line = AudioSegment.from_mp3('rain.mp3')
vocal_line = AudioSegment.from_mp3('you-spend-time-with-your-family.mp3')

In [430]:
music = mix_lines(
    drums_line,
    piano_line,
    noise_line,
    vocal_line,
    music_len=60
)

In [431]:
music.export('generated_music.mp3')

<_io.BufferedRandom name='generated_music.mp3'>

In [432]:
music